# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xl
import requests
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
## Transform the data as a json object
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': '/N TCQEAEL LAGEN(S O)BKLRGTMASDAS',
 'calculationPrice': 'close',
 'open': 138.84,
 'openTime': 1687869454697,
 'openSource': 'oialcffi',
 'close': 142.13,
 'closeTime': 1651933235014,
 'closeSource': 'flfaioic',
 'high': 142.13,
 'highTime': 1666230535699,
 'highSource': '1ae  mtylurnedeciidp5e ',
 'low': 138.84,
 'lowTime': 1691683534378,
 'lowSource': 'dmp ede n51lui eaecriyt',
 'latestPrice': 142.37,
 'latestSource': 'Close',
 'latestTime': 'July 2, 2021',
 'latestUpdate': 1702421790328,
 'latestVolume': 81255584,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 144.18,
 'delayedPriceTime': 1627198861388,
 'oddLotDelayedPrice': 140.86,
 'oddLotDelayedPriceTime': 1704449139643,
 'extendedPrice': 142.45,
 'extendedChange': -0.11,
 'extendedChangePercent': -0.0008,
 'extendedPriceTime': 1638743605999,
 'previousClose': 138.88,
 'previousVolume': 53225777,
 'change': 2.79,

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
## Building a data frame for our project
my_cols = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
df = pd.DataFrame(columns=my_cols)
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [13]:
df.append(
    pd.Series([
        symbol,
        price,
        market_cap,
        'N/A'
    ],
        index = my_cols
    ),
    ignore_index =True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,144.08,2299062045362,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [19]:
df = pd.DataFrame(columns=my_cols)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    df = df.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
        index = my_cols
    ),
    ignore_index =True
)

In [21]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,143.23,2298878393486,N/A
1,AAL,140.77,2405121665978,N/A
2,AAP,138.29,2330771589555,N/A
3,AAPL,140.68,2354745596071,N/A
4,ABBV,141.78,2292626871116,N/A
...,...,...,...,...
500,YUM,137.74,2343556854077,N/A
501,ZBH,139.67,2315247034373,N/A
502,ZBRA,143.88,2364204222231,N/A
503,ZION,139.32,2296190216628,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [7]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [26]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
df = pd.DataFrame(columns=my_cols)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
                index = my_cols
            ),
            ignore_index =True
        )
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,149.16,45823692832,N/A
1,AAL,22.10,13786646100,N/A
2,AAP,217.96,14063714489,N/A
3,AAPL,144.59,2445892902380,N/A
4,ABBV,116.64,211071596223,N/A
...,...,...,...,...
500,YUM,117.31,35544257651,N/A
501,ZBH,165.93,34137509440,N/A
502,ZBRA,560.70,28987082625,N/A
503,ZION,54.95,9016220377,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [27]:
portfolio_size = input('What is the value of your portfolio? ')

try:
    portfolio_size = float(portfolio_size)
except ValueError:
    print('Please enter a valid number, try again...')
    portfolio_size = float(input('What is the value of your portfolio? '))
    

What is the value of your portfolio? 1000000


In [28]:
position_size = portfolio_size/len(df.index)
print(position_size)
for i in range(0, len(df.index)):
    df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/df.loc[i,'Stock Price'])
    
df

1980.1980198019803


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,149.16,45823692832,13
1,AAL,22.10,13786646100,89
2,AAP,217.96,14063714489,9
3,AAPL,144.59,2445892902380,13
4,ABBV,116.64,211071596223,16
...,...,...,...,...
500,YUM,117.31,35544257651,16
501,ZBH,165.93,34137509440,11
502,ZBRA,560.70,28987082625,3
503,ZION,54.95,9016220377,36


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [38]:
writer = pd.ExcelWriter('recommended_trades_for_day.xlsx', engine='xlsxwriter')
df.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [39]:
# Color scheme for the excel file.
background_color = '#0A0A23'
font_color = 'ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [40]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [42]:
columns_format = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in columns_format.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, columns_format[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', columns_format[column][0], columns_format[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()